In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re

ua=UserAgent()

In [9]:
request = requests.get(
    "https://library.ceu.edu/ceu-library/electronic-theses-and-dissertations-etds/",
    headers={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    },
)

In [14]:
soup=BeautifulSoup(request.content)

In [20]:
departments = soup.find(
    "div", {"class": "et_pb_module et_pb_accordion et_pb_accordion_0"}
)

In [89]:
records = []
for dep in departments.find_all("div", {"class": "et_pb_accordion_item"}):
    records += [
        {
            "department": dep.find("h5").get_text(),
            "text": i.get_text(),
            "links": i.find_all("a"),
        }
        for i in dep.find_all("li")
    ]
df_content = pd.DataFrame.from_records(records)

In [99]:
full_content = (
    pd.DataFrame(
        df_content.set_index(["department", "text"]).links.apply(pd.Series).stack()
    )
    .reset_index()
    .drop(["level_2"], axis=1)
)

In [120]:
full_content["program"] = full_content["text"].apply(
    lambda x: x.strip().split(" ")[0].split(":")[0]
)
full_content = full_content.drop(["text"], axis=1)

In [121]:
full_content['program'].value_counts()

MA          148
MSc          28
MESPOM       13
PhD          13
Cultural      5
Doctoral      5
Business      2
Name: program, dtype: int64

#### Dropping doctoral school, because it's the same as the department phd all year

In [135]:
full_content = full_content.loc[
    full_content.department != "Doctoral School",
]
full_content=full_content.rename({0:"lib"},axis=1)

In [139]:
full_content['lib_link']=full_content['lib'].apply(lambda x: x.get('href'))
full_content['year']=full_content['lib'].apply(lambda x: x.get_text())
full_content.drop("lib",axis=1, inplace=True)

In [140]:
full_content

,department,program,lib_link,year
0,Economics,MA,http://tinyurl.com/gplxltt,2007
1,Economics,MA,http://tinyurl.com/gvl9sme,2008
2,Economics,MA,http://tinyurl.com/jzftq5t,2009
3,Economics,MA,http://tinyurl.com/j5zcywm,2010
4,Economics,MA,http://tinyurl.com/hu8k6l7,2011
...,...,...,...,...
204,Sociology,MSc,http://tinyurl.com/socmsc2017,2017
205,Sociology,MSc,https://tinyurl.com/socma2018,2018
206,Sociology,MSc,https://tinyurl.com/socma2019,2019
207,Sociology,MSc,https://tinyurl.com/socma2020,2020


In [172]:
def getpages(link):
    try:
        page=BeautifulSoup(requests.get(link.replace('+',"")).content)
        num=len([page.get_text() for page in page.find("td",{"class": "browsePager"}).find_all('a')][:-1])
    except AttributeError:
        num=0
    except: 
        print(link)
        num=0
    return 1+num

In [2]:
from tqdm.notebook import tqdm
tqdm.pandas()

/home/jbiro/.local/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [173]:
full_content['pages_on_lib']=full_content.lib_link.progress_apply(lambda x: getpages(x))

http://tinyurl.co+m/j87lccu



In [176]:
page=BeautifulSoup(requests.get(full_content.lib_link[12]).content)

In [177]:
[page.get_text() for page in page.find("td",{"class": "browsePager"}).find_all('a')][:-1]

['2']

In [192]:
def theses_link_crawler(link):
    page = BeautifulSoup(requests.get(link).content)
    titles=page.find_all(
        "span", {"class": "briefcitTitle"}
    )
    try:
        pages=[x.get('href') for x in page.find("td",{"class": "browsePager"}).find_all('a')][:-1]
        print("more page found at ", link)
    except:
        pages=[]
    
    links=[x.find("a").get("href") for x in titles]
    if len(pages)!=0:
        for more in pages:
            links+=[x.find("a").get("href") for x in BeautifulSoup(requests.get("https://sierra.ceu.edu/"+more).content).find_all(
        "span", {"class": "briefcitTitle"}
    )]
    return links
        
    

In [194]:
full_content["theses_links_all"]=full_content.lib_link.progress_apply(lambda x: theses_link_crawler(x.replace('+','')))

more page found at  https://tinyurl.com/ecoma2018
more page found at  https://tinyurl.com/ecoma2019
more page found at  http://tinyurl.com/ycayjpm
more page found at  http://tinyurl.com/legalm2008
more page found at  http://tinyurl.com/legalm2010
more page found at  http://tinyurl.com/legalm2012
more page found at  http://tinyurl.com/legalm2013
more page found at  http://tinyurl.com/legalma2014
more page found at  http://tinyurl.com/legalma2015
more page found at  http://tinyurl.com/legalma2016
more page found at  http://tinyurl.com/legalma2017
more page found at  https://tinyurl.com/legalma2018
more page found at  https://tinyurl.com/legalma2020
more page found at  http://tinyurl.com/cysrae
more page found at  http://tinyurl.com/c9f9lq
more page found at  http://tinyurl.com/gp2hv3f
more page found at  http://tinyurl.com/grxlbt5
more page found at  http://tinyurl.com/asr6lx



In [196]:
full_content['number_of_thesis']=full_content.theses_links_all.apply(lambda x: len(x))

In [203]:
full_content

,department,program,lib_link,year,pages_on_lib,theses_links_all,number_of_thesis
0,Economics,MA,http://tinyurl.com/gplxltt,2007,1,[/search?/X+%22CEU+Economics+Department+master...,35
1,Economics,MA,http://tinyurl.com/gvl9sme,2008,1,[/search?/X+%22CEU+Economics+Department+master...,33
2,Economics,MA,http://tinyurl.com/jzftq5t,2009,1,[/search?/X+%22CEU+Economics+Department+master...,36
3,Economics,MA,http://tinyurl.com/j5zcywm,2010,1,[/search?/X+%22CEU+Economics+Department+master...,38
4,Economics,MA,http://tinyurl.com/hu8k6l7,2011,1,[/search?/X+%22CEU+Economics+Department+master...,38
...,...,...,...,...,...,...,...
204,Sociology,MSc,http://tinyurl.com/socmsc2017,2017,1,[/search?/X%22CEU+Sociology+Department+master+...,28
205,Sociology,MSc,https://tinyurl.com/socma2018,2018,1,[/search?/X%22CEU+Sociology+Department+master+...,22
206,Sociology,MSc,https://tinyurl.com/socma2019,2019,1,[/search?/X%22CEU+Sociology+Department+master+...,27
207,Sociology,MSc,https://tinyurl.com/socma2020,2020,1,[/search?/X%22CEU+Sociology+Department+master+...,27


In [266]:
theses_full_df = (
    pd.DataFrame(
        full_content.drop(['pages_on_lib'],axis=1).set_index(["department", "program","lib_link","year","number_of_thesis"]).theses_links_all.apply(pd.Series).stack()
    )
    .reset_index()
    .drop(["level_5"], axis=1).rename({0:"thesis_link","number_of_thesis":"thesis_in_that_year"},axis=1)
)

,department,program,lib_link,year,thesis_in_that_year,thesis_link
0,Economics,MA,http://tinyurl.com/gplxltt,2007,35,/search?/X+%22CEU+Economics+Department+master+...
1,Economics,MA,http://tinyurl.com/gplxltt,2007,35,/search?/X+%22CEU+Economics+Department+master+...
2,Economics,MA,http://tinyurl.com/gplxltt,2007,35,/search?/X+%22CEU+Economics+Department+master+...
3,Economics,MA,http://tinyurl.com/gplxltt,2007,35,/search?/X+%22CEU+Economics+Department+master+...
4,Economics,MA,http://tinyurl.com/gplxltt,2007,35,/search?/X+%22CEU+Economics+Department+master+...
...,...,...,...,...,...,...
5820,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...
5821,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...
5822,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...
5823,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...


In [277]:
theses_df=theses_full_df.loc[theses_full_df['year'].isin(['2018','2019','2020','PhD (all years)',' PhD (all years)']),]
theses_df

,department,program,lib_link,year,thesis_in_that_year,thesis_link
396,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,/search?/X(+%22CEU+Economics+Department+master...
397,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,/search?/X(+%22CEU+Economics+Department+master...
398,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,/search?/X(+%22CEU+Economics+Department+master...
399,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,/search?/X(+%22CEU+Economics+Department+master...
400,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,/search?/X(+%22CEU+Economics+Department+master...
...,...,...,...,...,...,...
5820,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...
5821,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...
5822,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...
5823,Sociology,PhD,http://tinyurl.com/kqqlj35,PhD (all years),36,/search?/X%22+Sociology+Department%22+PhD&b=&l...


In [278]:
for index,row in tqdm(theses_df.iterrows(), total=theses_df.shape[0]):
    
    page=BeautifulSoup(requests.get("https://sierra.ceu.edu"+row['thesis_link'],headers={'User-Agent':str(ua.random)}).content)
    
    try:
        theses_df.loc[index,"thesis_link"]="https://sierra.ceu.edu"+page.find(id='recordnum').get('href')
    except:
        # if there is no new permanent link in the database we keep the original query
        pass
    
    try:
        theses_df.loc[index,"pdf_link"]=[s.get('href') for s in page.find('table',{"class":"bibLinks"}).find_all('a') if ".pdf" in s.get('href')][0]
    except:
        print("no pdf link: ",row['thesis_link'])
        
    theses_df.loc[index,"data_dict"]=str(dict(
    zip(
        pd.Series(page.find_all("td", {"class": "bibInfoLabel"})).apply(
            lambda x: x.get_text()
        ),
        pd.Series(page.find_all("td", {"class": "bibInfoData"})).apply(
            lambda x: x.get_text().strip()
        ),
    )
))

/home/jbiro/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/jbiro/.local/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


no pdf link:  /search?/X(%22CEU+Gender+Studies+Department+master+theses%22)&Da=2018&Db=2018&SORT=AX&m=v/X(%22CEU+Gender+Studies+Department+master+theses%22)&Da=2018&Db=2018&SORT=AX&m=v&SUBKEY=(%22CEU+Gender+Studies+Department+master+theses%22)/1%2C29%2C29%2CB/frameset&FF=X(%22CEU+Gender+Studies+Department+master+theses%22)&Da=2018&Db=2018&SORT=AX&m=v&27%2C27%2C
no pdf link:  /search?/X(%22CEU+Gender+Studies+Department+master+theses%22)&Da=2020&Db=2020&SORT=AX&m=v/X(%22CEU+Gender+Studies+Department+master+theses%22)&Da=2020&Db=2020&SORT=AX&m=v&SUBKEY=(%22CEU+Gender+Studies+Department+master+theses%22)/1%2C26%2C26%2CB/frameset&FF=X(%22CEU+Gender+Studies+Department+master+theses%22)&Da=2020&Db=2020&SORT=AX&m=v&15%2C15%2C
no pdf link:  /search?/X%22CEU+Gender+Studies+Department+PhD+theses%22&b=&l=&Da=&Db=&p=&m=v&SORT=D/X%22CEU+Gender+Studies+Department+PhD+theses%22&b=&l=&Da=&Db=&p=&m=v&SORT=D&SUBKEY=%22CEU+Gender+Studies+Department+PhD+theses%22/1%2C37%2C37%2CB/frameset&FF=X%22CEU+Gender+

no pdf link:  /search?/X+%22CEU+Political+Science+Department+master+theses%22&b=&l=&Da=2020&Db=2020&p=&m=v&SORT=A/X+%22CEU+Political+Science+Department+master+theses%22&b=&l=&Da=2020&Db=2020&p=&m=v&SORT=A&SUBKEY=+%22CEU+Political+Science+Department+master+theses%22/1%2C42%2C42%2CB/frameset&FF=X+%22CEU+Political+Science+Department+master+theses%22&b=&l=&Da=2020&Db=2020&p=&m=v&SORT=A&36%2C36%2C
no pdf link:  /search?/X(%22CEU+School+of+Public+Policy+master+theses%22)&Da=2018&Db=2018&SORT=AX/X(%22CEU+School+of+Public+Policy+master+theses%22)&Da=2018&Db=2018&SORT=AX&SUBKEY=(%22CEU+School+of+Public+Policy+master+theses%22)/1%2C41%2C41%2CB/frameset&FF=X(%22CEU+School+of+Public+Policy+master+theses%22)&Da=2018&Db=2018&SORT=AX&37%2C37%2C
no pdf link:  /search?/X%22CEU+Sociology+Department+master+theses%22&b=&l=&Da=2018&Db=2018&p=&m=v&SORT=A/X%22CEU+Sociology+Department+master+theses%22&b=&l=&Da=2018&Db=2018&p=&m=v&SORT=A&SUBKEY=%22CEU+Sociology+Department+master+theses%22/1%2C22%2C22%2CB/frame

theses_df.loc[0,"thesis_link"]="/search?/X+%22CEU+Economics+Department+master+theses%22&b=&l=&Da=2007&Db=2007&p=&m=v&SORT=A/X+%22CEU+Economics+Department+master+theses%22&b=&l=&Da=2007&Db=2007&p=&m=v&SORT=A&SUBKEY=+%22CEU+Economics+Department+master+theses%22/1%2C35%2C35%2CB/frameset&FF=X+%22CEU+Economics+Department+master+theses%22&b=&l=&Da=2007&Db=2007&p=&m=v&SORT=A&1%2C1%2C"

In [312]:
final=pd.concat([theses_df.drop(['data_dict'],axis=1).reset_index(drop=True),pd.json_normalize(theses_df.data_dict.apply(lambda x: eval(x)))],axis=1)

final.subject

In [314]:
final.to_csv('final_almost.csv')

In [6]:
theses_full=pd.read_csv('backup_full.csv')

In [10]:
theses_full.shape

(5825, 6)

In [12]:
final.head()

,department,program,lib_link,year,thesis_in_that_year,thesis_link,pdf_link,Author,Title,Published,...,Series,Language,Summary,Note,File Type,Local note,Access,System Det,Subject,Alt Title
0,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,https://sierra.ceu.edu/record=b1318696,http://www.etd.ceu.edu/2018/esendi_lynda.pdf,"Esendi, Lynda Frida",Alleviating strike actions in the healthcare s...,"Budapest : Central European University, 2018",...,CEU Economics Department master theses ; 2018/1,English,A review of the Kenya healthcare system shows ...,Degree: MA,PDF file (1524k),ETD,Unrestricted,System requirements: Adobe Acrobat Reader; Pos...,NaN,NaN
1,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,https://sierra.ceu.edu/record=b1410971,http://www.etd.ceu.edu/2018/chytil_mario.pdf,"Chytil, Marió",Analysis of deviations of reticuloruminal pH d...,"Budapest : Central European University, 2018",...,CEU Economics Department master theses ; 2018/51,English,My Capstone Project is aimed at describing the...,Degree: MSc,PDF file (166k),ETD,Capstone project,Unrestricted,NaN,NaN
2,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,https://sierra.ceu.edu/record=b1410983,http://www.etd.ceu.edu/2019/ronai_dora.pdf,"R贸nai, D贸ra",Asset audit and compliance in FAO / D贸ra R贸nai,"Budapest : Central European University, 2018",...,CEU Economics Department master theses ; 2018/44,English,Asset Audit and Compliance in FAO,Degree: MSc,PDF file (566k),ETD,Capstone project,Unrestricted,NaN,NaN
3,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,https://sierra.ceu.edu/record=b1410974,http://www.etd.ceu.edu/2018/menyhert_kristof.pdf,"Menyhért, Kristóf",An attempt to optimize a questionnaire-based r...,"Budapest : Central European University, 2018",...,CEU Economics Department master theses ; 2018/42,English,An attempt to optimize a questionnaire-based r...,Degree: MSc,PDF file (860k),ETD,Capstone project,Unrestricted,NaN,NaN
4,Economics,MA,https://tinyurl.com/ecoma2018,2018,52,https://sierra.ceu.edu/record=b1399987,http://www.etd.ceu.edu/2018/nazaruk_anna.pdf,"Nazaruk, Anna",Bitcoin 2017 : was there a bubble? / Anna Nazaruk,"Budapest : Central European University, 2018",...,CEU Economics Department master theses ; 2018/29,English,This research examines existence of Bitcoin bu...,Degree: MA,PDF file (669k),ETD,Unrestricted,System requirements: Adobe Acrobat Reader; Pos...,NaN,NaN


In [11]:
final=final.drop(['Unnamed: 0'],axis=1)

In [19]:
final['thesis_link'].drop_duplicates().shape

(1727,)

In [23]:
final.groupby(['program']).agg(n=('year','size'))

,n
program,
Business,40
Cultural,24
MA,948
MESPOM,41
MSc,129
PhD,546


In [24]:
948+24+41+129

1142

In [27]:
ma_sample=final.loc[final.program.isin(['MA','MSc','MESPOM','Cultural'])].sample(n=100,random_state=20201218)

In [30]:
phd_sample=final.loc[final.program.isin(['PhD'])].sample(n=30,random_state=20201218)

In [32]:
business_sample=final.loc[final.program.isin(['Business'])].sample(n=10,random_state=20201218)

In [34]:
ma_sample.to_csv('MA_100.csv',index=False)

In [35]:
phd_sample.to_csv('PHD_30.csv',index=False)

In [36]:
business_sample.to_csv('Business_10.csv',index=False)